In [1]:
from google import genai
import os
from dotenv import load_dotenv
from utils.constants import TRANSCRIPT_PROMPT, SYSTEM_PROMPT,  FUNCTION_CALL_CONFIG
from services.firestore_service import FirestoreService
from services.tools import get_transcript_from_url, youtube_search


load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=GEMINI_API_KEY)


class YoutubeService:
    def __init__(self, session_id: str, user_id: str, tag: str):
        self.session_id = session_id
        self.user_id = user_id
        self.tag = tag
        self.firestore = FirestoreService(session_id=self.session_id, tag=self.tag)


    async def get_youtube_summary (self, query: str):

        await self.firestore.store_conversation(self.user_id, query)
        recent_conversations = await self.firestore.get_user_conversations(limit=10)
   
        conversation_context = []
        for conv in recent_conversations:
            conversation_context.extend([
                {"role": "user", "content": conv['message']},
                {"role": "assistant", "content": conv['response']}
            ])

        first_response = client.models.generate_content(
            model="gemini-1.5-pro",
            config=FUNCTION_CALL_CONFIG,
            contents=f"{SYSTEM_PROMPT}\n\n Past Conversations: {conversation_context}\n\n User Query: {query}" 
        )

        part = first_response.candidates[0].content.parts[0]

        function = part.function_call


        if function is not None:
            available_functions = {
                "youtube_search": youtube_search,
                "get_transcript_from_url": get_transcript_from_url,
            }

            function_to_call = available_functions.get(function.name, None)
            function_response = function_to_call(**function.args)

            print("#################")

            print(function_response)

            if function.name == "get_transcript_from_url":
                second_response = client.models.generate_content(
                    model="gemini-1.5-pro",
                    config=FUNCTION_CALL_CONFIG,
                    contents=f"{TRANSCRIPT_PROMPT}\n\n Past Conversations: {conversation_context}\n\n video_info: {function_response}" 
                )

                print(second_response)

                await self.firestore.store_conversation(self.user_id, query, second_response.text)

                return second_response.text
            else:
                print(function_response)

                return function_response
                
        else:

            return part.text

In [2]:
youtube_service = YoutubeService(session_id="hgc436364723feqfrgrq", user_id="general", tag="general")
print(await youtube_service.get_youtube_summary("https://www.youtube.com/watch?v=bslcx4LRFL0&t=1s"))

#################
{'transcript': "all I'm asking for is $2,000 you and I know $2,000 is not really it's not a big deal you have it I know you know I know you have it all right so don't just don't be stingy just Lord I should remove money from my account oo listen babe don't make me commit M Davis leave you know what take of everything I have bought you baby you are overreacting necklace shirt your watch your hand chain your shoes in fact your boxes take it off babe now you're not serious right yes I saw the edits they are sloppy m- revert to version two and align with the brand guide and this time tell Margaret to Pro it before you give it to me okay have we finalized with the vendor for the spring campaign no what why why is my inbox still empty by 3:00 p.m. please thank you people are so incompetent com in hi M TR hi um the clients they just sent their feedback they said they don't like the ad placement I don't understand show me so this is what we're going to do we're going to switc

In [3]:
youtube_service = YoutubeService(session_id="hgc436364723feqfrgrq", user_id="general", tag="general")
print(await youtube_service.get_youtube_summary("why did chioma not want to marry obiora?"))

The user in the past conversation was referring to Tra, not Chioma.  Tra initially hesitated to marry Obiora because his "evil" qualities - wealth, confidence, and a traditional mindset - clashed with her independent nature and past experiences with men.  She was afraid of becoming dependent on a man and losing her hard-earned independence.

